In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline

In [2]:
modelo=pd.read_csv('data/modelo.csv',low_memory=False)

FileNotFoundError: File b'data/modelo.csv' does not exist

In [ ]:
modelo.head()

In [ ]:
modelo.isnull().sum()

In [ ]:
modelo.drop(['url', 'sku', 'skus','channel','staticpage','condition','storage','color','search_engine','device_type','campaign_source','country','region','search_term','new_vs_returning','city','region','screen_resolution','operating_system_version','browser_version'], axis=1, inplace=True)

In [ ]:
training= pd.read_csv('data/training.csv',low_memory=False)

# Armado del modelo

In [ ]:
modelo.sort_values('person',inplace=True)

In [ ]:
import calendar

modelo['timestamp'] = pd.to_datetime(modelo['timestamp'])
modelo['dia']=modelo['timestamp'].dt.dayofyear


modelo['dias_distintos'] = modelo['person'].map(modelo.groupby(['person',(modelo['timestamp'].dt.month==5) ] )['dia'].nunique().unstack()[True])   #ultimo mes
modelo['dias_distintos']=modelo['dias_distintos'].fillna(0).astype(int)

modelo['ultima_visita']=modelo['person'].map(modelo.groupby('person' )['dia'].max())    
modelo['ultima_visita']=modelo['ultima_visita'].fillna(0).astype(int)
modelo.drop('dia',1,inplace=True)


modelo['modelos_distintos_vistos']= modelo['person'].map( modelo.groupby('person')['model'].nunique())
modelo.drop('model',1,inplace=True)


modelo['campaña_ultima_semana']=modelo['person'].map(modelo.groupby(['person',(modelo['timestamp'].dt.month==5) & (modelo['timestamp'].dt.day>23) & (modelo['event']=="ad campaign hit")]).size().unstack()[True]) 
modelo['campaña_ultima_semana']=modelo['campaña_ultima_semana'].fillna(0).astype(int)


modelo['busqueda_celular']=modelo['person'].map(modelo.groupby(['person',(modelo['timestamp'].dt.month==5) & (modelo['timestamp'].dt.day>23) & (modelo['event']=="searched products")]).size().unstack()[True]) 
modelo['busqueda_celular']=modelo['busqueda_celular'].fillna(0).astype(int)


modelo['total_visitas_usuario'] = modelo.groupby('person')['person'].transform('count')


modelo['visitas_ultimo_mes']=modelo['person'].map(modelo.groupby(['person',(modelo['timestamp'].dt.month)==5]).size().unstack()[True])
modelo['visitas_ultimo_mes']=modelo['visitas_ultimo_mes'].fillna(0).astype(int)


modelo['visitas_ultimos_15']=modelo['person'].map(modelo.groupby(['person', (modelo['timestamp'].dt.month==5)& (modelo['timestamp'].dt.day>15)]).size().unstack()[True])
modelo['visitas_ultimos_15']=modelo['visitas_ultimos_15'].fillna(0).astype(int)


modelo['visitas_ultima_semana']=modelo['person'].map(modelo.groupby(['person', (modelo['timestamp'].dt.month==5)& (modelo['timestamp'].dt.day>23)]).size().unstack()[True])
modelo['visitas_ultima_semana']=modelo['visitas_ultima_semana'].fillna(0).astype(int)


modelo['visitas_ultimos_3']=modelo['person'].map(modelo.groupby(['person', (modelo['timestamp'].dt.month==5)& (modelo['timestamp'].dt.day>27)]).size().unstack()[True])
modelo['visitas_ultimos_3']=modelo['visitas_ultimos_3'].fillna(0).astype(int)


modelo['total_checkout']=modelo['person'].map(modelo.groupby(['person','event']).size().unstack()['checkout'])
modelo['total_checkout']=modelo['total_checkout'].fillna(0).astype(int)


modelo['checkouts_ultimo_mes']=modelo['person'].map(modelo.groupby(['person','event',modelo['timestamp'].dt.month==5 ]).size().unstack()[True].unstack()['checkout'])   
modelo['checkouts_ultimo_mes']=modelo['checkouts_ultimo_mes'].fillna(0).astype(int)



modelo['checkouts_ultima_semana']=modelo['person'].map(modelo.groupby(['person','event',(modelo['timestamp'].dt.month==5)& (modelo['timestamp'].dt.day>23)]).size().unstack()[True].unstack()['checkout'])
modelo['checkouts_ultima_semana']=modelo['checkouts_ultima_semana'].fillna(0).astype(int)



modelo['total_compras']=modelo['person'].map(modelo.groupby(['person','event']).size().unstack()['conversion'])
modelo['total_compras']=modelo['total_compras'].fillna(0).astype(int)


modelo['compras_ultimo_mes']=modelo['person'].map(modelo.groupby(['person','event',modelo['timestamp'].dt.month==5 ]).size().unstack()[True].unstack()['conversion'])
modelo['compras_ultimo_mes']=modelo['compras_ultimo_mes'].fillna(0).astype(int)



modelo['compras_ultima_semana']=modelo['person'].map(modelo.groupby(['person','event',(modelo['timestamp'].dt.month==5)& (modelo['timestamp'].dt.day>23)]).size().unstack()[True].unstack()['conversion'])
modelo['compras_ultima_semana']=modelo['compras_ultima_semana'].fillna(0).astype(int)


modelo['subscripciones_ultimo_mes']=modelo['person'].map(modelo.groupby(['person','event',modelo['timestamp'].dt.month==5 ]).size().unstack()[True].unstack()['lead'])
modelo['subscripciones_ultimo_mes']=modelo['subscripciones_ultimo_mes'].fillna(0).astype(int)



In [ ]:
modelo=modelo.drop_duplicates(subset=['person'])
modelo.drop(['timestamp','event'], 1, inplace=True)

In [ ]:
modelo=pd.merge(modelo, training, on='person', how='left')

In [ ]:
modelo.head()

In [ ]:
a_predecir=modelo[modelo['label'].isnull()]
a_predecir.head()

In [ ]:
modelo.drop(modelo[modelo['label'].isnull()].index,inplace=True)
modelo['label']=modelo['label'].astype(int)
modelo.head()

In [ ]:
len(modelo[modelo['label']==1]),len(modelo[modelo['label']==0]) 

In [ ]:
len(modelo[modelo['label']==1])*100/len(modelo['label']),len(modelo[modelo['label']==0])*100/len(modelo['label'])

In [ ]:
modelo[modelo['label']==1].head()

In [ ]:
modelo[modelo['label']==0].head()

In [ ]:
modelo[modelo['label']==1].mean()-modelo[modelo['label']==0].mean()

## MACHINE LEARNING

In [ ]:
##Ahora empieza ML

from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score,precision_score,recall_score,roc_auc_score
from sklearn.model_selection import GridSearchCV,StratifiedKFold,RandomizedSearchCV

## Split

In [ ]:
X = modelo.drop(['person','label'], axis=1)
y = modelo['label']

In [ ]:
from sklearn.feature_selection import SelectKBest
model= SelectKBest(k=12).fit(X,y)
columnas=list(X.columns[model.get_support()])
columnas

In [ ]:
model=model.transform(X)

In [ ]:
X = pd.DataFrame.from_records(model)
X.columns=columnas

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=42,test_size=0) 

In [ ]:
len(X_train),len(X_test),len(y_train),len(y_test)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

## Algoritmos de clasificación

## XGBOOST

In [ ]:
import xgboost as xgb
my_classifier1 = xgb.XGBClassifier(objective ='binary:logistic',max_depth =1,learning_rate = 0.02, n_estimators = 500,scale_pos_weight=8)
my_classifier1.fit(X_train,y_train)
#predictions=my_classifier1.predict(X_test)
#metrics.confusion_matrix(y_test, predictions)

In [ ]:
#accuracy_score(y_test, predictions),roc_auc_score(y_test,  my_classifier1.predict_proba(X_test)[:,1])

## RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier #a mayor depth, mas precisos los 0, menos los 1,class weight cambia mucho
my_classifier2 = RandomForestClassifier(criterion= 'entropy',max_depth= 5,n_estimators=1000, max_features= 'auto', class_weight={1:9} ,n_jobs=-1)
my_classifier2.fit(X_train,y_train)
#predictions=my_classifier2.predict(X_test)
#metrics.confusion_matrix(y_test, predictions)

In [ ]:
#accuracy_score(y_test, predictions),roc_auc_score(y_test,  my_classifier2.predict_proba(X_test)[:,1])

## Voting classifier

In [ ]:
from sklearn.ensemble import VotingClassifier
vclf = VotingClassifier(estimators=[  ('XGB', my_classifier1), ('RF', my_classifier2)], voting='soft', n_jobs =-1)
vclf = vclf.fit(X_train, y_train)
#predictions = vclf.predict(X_test)
#metrics.confusion_matrix(y_test, predictions)

In [ ]:
#accuracy_score(y_test, predictions),roc_auc_score(y_test,  vclf.predict_proba(X_test)[:,1])

## Random search

In [ ]:
"""
vclf = VotingClassifier(estimators=[  ('XGB', xgb.XGBClassifier(objective ='binary:logistic' , max_depth =1,learning_rate = 0.01, n_estimators = 400 )),
                                    ('RF', RandomForestClassifier(n_jobs=-1,max_features= 'auto',  max_depth= 5,n_estimators=1000  ))], voting='soft', n_jobs =-1)

params = {'XGB__scale_pos_weight': [i for i in range(1,10)],
        'RF__class_weight': [  {1: i for i in range(1,10)}],
        'RF__criterion': ['gini','entropy' ], 
         }

folds = 10
param_comb = 20

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(vclf, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=-1, cv=skf.split(X_train,y_train), verbose=3, random_state=1001 )
random_search.fit(X_train, y_train)
print('\n Best hyperparameters:')
print(random_search.best_params_)
resultados=pd.DataFrame(random_search.cv_results_)
resultados=resultados.sort_values('mean_test_score',ascending=False)
resultados.head()
"""

## KAGGLE PREDICTIONS

In [ ]:
kaggle= pd.read_csv('data/kaggle.csv',low_memory=False)
kaggle=pd.merge(kaggle, a_predecir, on='person', how='inner')

In [ ]:
a_predecir.head()

In [ ]:
kaggle2=kaggle.loc[:,columnas]
kaggle2.head()

In [ ]:
predictions=vclf.predict_proba(kaggle2)

In [ ]:
kaggle['prediction0'] = predictions[:,0]
kaggle['label'] = predictions[:,1]

In [ ]:
kaggle.head()

In [ ]:
kaggle[['person','label']].to_csv('predicciones4.csv', encoding='utf-8', index=False)